
# PRACTICA NLP. NOTEBOOK 2

## 2. Etapa de preprocesado de texto:

El alumno preparará una etapa de preprocesado de reviews que permita adecuar el formato de las mismas a uno más adecuado. Será la etapa previa al entrenamiento del modelo de sentimiento.

Todo el preprocesado deberá incluirse en una función de Python que contenga todo el procesado de texto. Esta función puede (es recomendable) contener otras funciones que realicen tareas más concretas (eliminar stopwords, eliminar signos de puntuación, etc.).

In [1]:
import pandas as pd
import sys
sys.path.append('../..')
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import spacy
from spacy import displacy
#%matplotlib inline al importar este notebook en el 3 no le gusta esta línea...

In [2]:
#En el notebook anterior he guardado una copia del dataframe creado con las reviews de los instrumentos musicales. 
#lo cargo en este notebook

df = pd.read_csv('df_music_inst.csv', sep=';')

In [3]:
print(df.head(2))
df.info()

   overall                                             review
0      1.0                 It's good for beginners Five Stars
1      1.0  I recommend this starter Ukulele kit.  I has e...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   overall  60000 non-null  float64
 1   review   60000 non-null  object 
dtypes: float64(1), object(1)
memory usage: 937.6+ KB


In [4]:
#Empiezo preparando datos para realizar pruebas con spacy:
texts = df['review'].str.lower()
texts= texts.str.split()
reviews = df['review'].str

In [5]:
#Realizo pruebas con distintos tamaños y finalmente confirmo que se me carga el dataset completo. 
#texts= texts[:1000]
#reviews= reviews[:1000]
#df_reviews=df['review'][:1000]
df_review=df['review'][1] #prueba individual

In [6]:
df_rev= df['review'].tolist()
print(type(texts))
print(type(df_rev))
df_rev[1]

<class 'pandas.core.series.Series'>
<class 'list'>


'I recommend this starter Ukulele kit.  I has everything you\nneed to learn the Ukulele. Five Stars'

In [7]:
#instanciar modulo spacy en inglés
import en_core_web_sm
nlp = spacy.load('en_core_web_sm', exclude=['ner', 'parser'])

In [8]:
nlp.pipeline #confirmo qué hay en el pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1c07ffe3860>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1c069ea48b0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1c0000644c0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1c00005f180>)]

In [9]:
pipe_analysis = nlp.analyze_pipes(pretty=True) #overview para descartar errores


============================= Pipeline Overview =============================

#   Component         Assigns       Requires   Scores      Retokenizes
-   ---------------   -----------   --------   ---------   -----------
0   tok2vec           doc.tensor                           False      
                                                                      
1   tagger            token.tag                tag_acc     False      
                                                                      
2   attribute_ruler                                        False      
                                                                      
3   lemmatizer        token.lemma              lemma_acc   False      

✔ No problems found.


In [10]:
#Empiezo por preparar una función sencilla con elementos de dentro y fuera de spacy
import string
punct= string.punctuation

In [11]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
#cojo los stopwords de un proyecto de internet que me ha gustado mucho..quizá demasiado correcto para las reviews, si tengo tiempo buscaré algo más coloquial. 
with open("stopwords.txt","r") as text:
    stopwords= text.read()
    print(stopwords)

'd
'll
'm
're
's
've
0
000
1
10
2
3
4
5
6
7
8
9
a
about
above
across
after
afternoon
afterwards
again
against
all
almost
alone
along
already
also
although
always
am
among
amongst
amount
an
and
another
any
anyhow
anyone
anything
anyway
anywhere
app
are
around
as
ask
asked
asking
asks
at
back
be
became
because
become
becomes
becoming
been
before
beforehand
behind
being
below
beside
besides
between
beyond
both
bottom
brother
but
by
ca
call
called
calling
calls
came
can
cannot
com
come
comes
coming
comment
commented
commenting
comments
could
couldn
daughter
day
days
did
didn
do
does
doesn
doing
don
done
down
dr
dr.
due
during
each
edt
eight
either
eleven
else
elsewhere
empty
enough
est
even
evening
ever
every
everyone
everything
everywhere
except
facebook
family
father
feel
feeling
feels
felt
few
fifteen
fifty
find
finding
finds
first
five
for
former
formerly
forty
found
four
friday
friend
from
front
full
further
gave
get
gets
getting
give
given
gives
giving
gmail
go
goes
going
gone
google

In [13]:
# función básica con spacy para data_cleaning: minúsculas, lemma, punct y stopwords. 
# creo que esta es sencilla y válida para mi preprocesado del siguiente notebook..

clean_tokens=[]
def text_cleaning(sentence):
    doc=nlp(sentence)
    tokens=[]
    for token in doc:
        if token.lemma_ !="-PRON-":
            temp= token.lemma_.lower().strip()
        else:
            temp= token.lower_
        tokens.append(temp)
    
    clean_tokens=[]
    for token in tokens:
        if token not in stopwords and token not in punct:
            clean_tokens.append(token)
    return clean_tokens

In [14]:
text_cleaning(df_rev[3])
text_cleaning(df['review'][17])

['buy',
 'starter',
 'kit',
 'learn',
 'play',
 'money',
 'great',
 'start',
 'complete',
 'component',
 'construct',
 'good',
 'quality',
 'money']

In [15]:
type(clean_tokens)

list

In [16]:
#otras pruebas hechas para investigar preprocesado: 

In [17]:
#ejemplo con 1 review
doc = nlp(df_review)

#Creating and updating our list of tokens using list comprehension 
tokens = [token.text for token in doc]
print(tokens)

['I', 'recommend', 'this', 'starter', 'Ukulele', 'kit', '.', ' ', 'I', 'has', 'everything', 'you', '\n', 'need', 'to', 'learn', 'the', 'Ukulele', '.', 'Five', 'Stars']


In [18]:
tokenized_text = pd.DataFrame()

for i, token in enumerate(doc):
    tokenized_text.loc[i, 'text'] = token.text
    tokenized_text.loc[i, 'lemma'] = token.lemma_,
    tokenized_text.loc[i, 'pos'] = token.pos_
    tokenized_text.loc[i, 'tag'] = token.tag_
    tokenized_text.loc[i, 'dep'] = token.dep_
    tokenized_text.loc[i, 'shape'] = token.shape_
    tokenized_text.loc[i, 'is_alpha'] = token.is_alpha
    tokenized_text.loc[i, 'is_stop'] = token.is_stop
    tokenized_text.loc[i, 'is_punctuation'] = token.is_punct

tokenized_text[:20]

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop,is_punctuation
0,I,I,PRON,PRP,,X,True,True,False
1,recommend,"(recommend,)",VERB,VBP,,xxxx,True,False,False
2,this,"(this,)",DET,DT,,xxxx,True,True,False
3,starter,"(starter,)",ADJ,JJ,,xxxx,True,False,False
4,Ukulele,"(Ukulele,)",PROPN,NNP,,Xxxxx,True,False,False
5,kit,"(kit,)",NOUN,NN,,xxx,True,False,False
6,.,"(.,)",PUNCT,.,,.,False,False,True
7,,"( ,)",SPACE,_SP,,,False,False,False
8,I,"(I,)",PRON,PRP,,X,True,True,False
9,has,"(have,)",VERB,VBZ,,xxx,True,True,False


In [19]:
#pruebo con la opción de spacy para muchos textos creando una lista: 
docs= list(nlp.pipe(df_rev))

In [20]:
for doc in docs[0]:
    print(doc)

It
's
good
for
beginners
Five
Stars


In [21]:
for token in docs[2]:
    # Imprimo Token, lemma y part-of-speech tag
    print('{0:10}{1:10}{2:10}'.format(token.text, token.lemma_, token.pos_))

G'daughterg'daughterNOUN      
received  receive   VERB      
this      this      DET       
for       for       ADP       
Christmas Christmas PROPN     
present   present   NOUN      
last      last      ADJ       
year      year      NOUN      
and       and       CCONJ     
plays     play      VERB      
if        if        SCONJ     
often     often     ADV       
.         .         PUNCT     
Learning  learn     VERB      
new       new       ADJ       
songs     song      NOUN      
to        to        PART      
play      play      VERB      
regularly regularly ADV       


In [22]:
displacy.render(docs[0], style='dep')

C:\Users\evaes\anaconda3\lib\site-packages\spacy\displacy\__init__.py:125: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  warnings.warn(Warnings.W005)


In [23]:
#Una vez hechos mis pinitos, he encontrado y retocado un preprocesado con oop con spacy que me parece muy completo..

import re
from typing import List

import spacy
from spacy.tokens import Doc
from tqdm import tqdm


class SpacyPreprocessor:
    def __init__(
        self,
        spacy_model=None,
        remove_numbers=False,
        remove_special=True,
        pos_to_remove=None,
        remove_stopwords=False,
        lemmatize=False,
    ):
        
        self._remove_numbers = remove_numbers # quitar o no quitar números.En este caso Falso
        self._pos_to_remove = pos_to_remove #qué part of speech quitar. En este caso ninguno
        self._remove_stopwords = remove_stopwords #lo mismo con el resto. stopwords
        self._remove_special = remove_special # caracteres especiales (incluidos números..)
        self._lemmatize = lemmatize

        if not spacy_model:
            self.model = spacy.load("en_core_web_sm")
        else:
            self.model = spacy_model

    @staticmethod
    def download_spacy_model(model="en_core_web_sm"):
        print(f"Downloading spaCy model {model}")
        spacy.cli.download(model)
        print(f"Finished downloading model")

    @staticmethod
    def load_model(model="en_core_web_sm"):
        return spacy.load(model, disable=["ner", "parser"])

    def tokenize(self, text) -> List[str]: #le digo que me devuelva una lista de str
        
        doc = self.model(text)
        return [token.text for token in doc]

    def preprocess_text(self, text) -> str: #aquí le pido que me lleve a cabo el pipeline de spacy, quite lo que no quiero y me devuelva texto limpio en str
        
        doc = self.model(text)
        return self.__clean(doc)

    def preprocess_text_list(self, texts=List[str]) -> List[str]: #pido el pipeline de spacy para limpiar una lista de textos y me devuelva una lista de textos limpios
        
        clean_texts = []
        for doc in tqdm(self.model.pipe(texts)):
            clean_texts.append(self.__clean(doc))

        return clean_texts

    def __clean(self, doc: Doc) -> str:

        tokens = []
        # Quitar part of speech tags (si es distinto de none)
        if self._pos_to_remove:
            for token in doc:
                if token.pos_ not in self._pos_to_remove:
                    tokens.append(token)
        else:
            tokens = doc

        # Quitar números (si está en True)
        if self._remove_numbers:
            tokens = [
                token for token in tokens if not (token.like_num or token.is_currency)
            ]

        # Quitar Stopwords
        if self._remove_stopwords:
            tokens = [token for token in tokens if not token.is_stop]
        # otros tokens no deseados
        tokens = [
            token
            for token in tokens
            if not (
                token.is_punct or token.is_space or token.is_quote or token.is_bracket
            )
        ]

        # tokens vacíos
        tokens = [token for token in tokens if token.text.strip() != ""]

        # Lematizar 
        if self._lemmatize:
            text = " ".join([token.lemma_ for token in tokens])
        else:
            text = " ".join([token.text for token in tokens])

        if self._remove_special:
            # Quitar caracteres no alphabeticos
            text = re.sub(r"[^a-zA-Z\']", " ", text)
        # quitar caracteres non-Unicode (incl.números)
        text = re.sub(r"[^\x00-\x7F]+", "", text)
        text = text.lower()
        return text

if __name__ == "__main__":
    spacy_model = SpacyPreprocessor.load_model()
    preprocessor = SpacyPreprocessor(spacy_model=spacy_model, lemmatize=True, remove_numbers=True, remove_stopwords=True, remove_special=True)
    clean_text = preprocessor.preprocess_text(df_rev[4])
    clean_texts = preprocessor.preprocess_text_list(df_rev) #cargo todo el documento (60000 reviews)
    print(clean_text)

60000it [04:24, 226.62it/s]

purchase gift friend want learn neat little set adore give star box smash good receive package big brown package box fine meaning product box instrument come damage ship luckily inside fine break company well care come ship damage item good starter kit


In [24]:
#para probar la clase anterior reduzco la lista de reviews
df_revs= df['review'].tolist()[:100]


In [25]:
# para clean_texts = preprocessor.preprocess_text_list(df_revs) el resultado es rápido (tarda 1 minuto) y parece muy positivo. 
#Vuelvo a probar con el texto completo pero imprimo sólo una parte.
print(clean_texts[:100])

#por lo que veo podría meter otra función para eliminar palabras de menos de 2 caracteres. 
#por lo demás tiene muy buena pinta. 


['good beginner star', 'recommend starter ukulele kit need learn ukulele star', "g'daughter receive christmas present year play learn new song play regularly", 'accord order history buy kit end black friday week lightning deal write nearly year later stringed instrument string need stretch stay tune day play minute tune play minute tune tune daily week uke hold tune great tuner include package bulk minus star real beginner way tune thing need tuner reference note piano keyboard download decent tuner app buck go need lot couple week firebrand ukulele laminated instrument unknown wood kind dead sounding soprano uke generally sound bit flattish distinctive character finish smooth spotty decal sound hole bubble fretboard barely adequately finish fret nicely flush edge neck feel smooth stay smooth despite temperature change bevel fret edge file especially polished snag catch fret dot inset deeply fingerboard paint upper edge neck nice touch action okay near headstock starter chord live tune

In [26]:
#guardo en un documento de python tanto la función sencilla como la clase para poder valorar cuál uso en el siguiente notebook. 
print(type(clean_text)) 
print(type(clean_texts))

<class 'str'>
<class 'list'>
